In [2]:
# %% [markdown]
# # 분류 모델 추론 스크립트 (Notebook Edition)
# * 로컬 `roberta/` 폴더에 fine-tuning 된 모델이 있다고 가정
# * 테스트 데이터: `./data/cls_test.json`
# * 결과는 `cls_output.json`으로 저장

# %% Imports & 환경설정
import os, json, torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM, pipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✔ Using device:", device)


✔ Using device: cuda


In [3]:
base_dir = os.getcwd()  # Jupyter에서는 __file__ 대신
roberta_path = "./roberta"
kogpt2_path = os.path.join(base_dir, "..", "kogpt2-finetuned")

label_map = {
    0: "졸업요건",
    1: "학교 공지사항",
    2: "학사일정",
    3: "식단 안내",
    4: "통학/셔틀 버스"
}

In [9]:
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_path)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_path).to(device).eval()

kogpt2_tokenizer = AutoTokenizer.from_pretrained(kogpt2_path)
kogpt2_model = AutoModelForCausalLM.from_pretrained(kogpt2_path).to(device)

generator = pipeline(
    "text-generation",
    model=kogpt2_model,
    tokenizer=kogpt2_tokenizer,
    device=0 if torch.cuda.is_available() else -1
)


Device set to use cuda:0


In [11]:
input_path = os.path.join(base_dir, "..", "data", "cls_test.json")
output_path = os.path.join(base_dir, "..", "outputs", "output.json")

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

questions = [entry.get("question", "").strip() for entry in data if entry.get("question", "").strip()]
print(f"✔ {len(questions)}개의 질문이 로드되었습니다.")

✔ 1000개의 질문이 로드되었습니다.


In [12]:
# 분류
inputs = roberta_tokenizer(questions, return_tensors="pt", padding=True, truncation=True).to(device)
with torch.no_grad():
    logits = roberta_model(**inputs).logits
    pred_ids = logits.argmax(dim=-1).cpu().tolist()
topics = [label_map.get(pred_id, "기타") for pred_id in pred_ids]

# 프롬프트
prompts = [f"[TOPIC] {topic} [USER] {question} [SEP] [BOT]" for topic, question in zip(topics, questions)]

# 답변 생성
generated_outputs = generator(
    prompts,
    max_new_tokens=50,
    num_return_sequences=1,
    do_sample=True,
    pad_token_id=kogpt2_tokenizer.eos_token_id
)


In [15]:
results = []
for question, prompt, topic, gen in zip(questions, prompts, topics, generated_outputs):
    generated_text = gen[0]["generated_text"]
    answer = generated_text.replace(prompt, "").strip()

    if not answer or len(answer) < 5:
        answer = "죄송합니다. 답변을 생성할 수 없습니다."

    results.append({
        "question": question,
        "topic": topic,
        "answer": answer
    })

# 저장
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"✅ 총 {len(results)}건 처리 완료 → {output_path}")


✅ 총 1000건 처리 완료 → D:\lastproject\Termproject_{9조}\src\..\outputs\output.json


In [17]:

# %% 결과 저장
out_path = os.path.join(base_dir, "cls_output.json")
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("📄 Saved predictions to:", out_path)

📄 Saved predictions to: D:\lastproject\Termproject_{9조}\src\cls_output.json
